In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Datascience/Fake News/train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data = data.dropna()

In [4]:
X = data.drop('label', axis =1) 
y = data['label']

In [5]:
X.shape, y.shape

((18285, 4), (18285,))

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [7]:
#Vocabulary size
voc_size = 5000

In [8]:
#Onehot Representation
messages = X.copy()
messages.reset_index(inplace = True)

In [9]:
#Data preprocessing
import nltk
import re
from nltk.corpus import stopwords


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [12]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [13]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[1720, 4228, 4064, 3481, 1072, 2355, 885, 1302, 1911, 1684],
 [4978, 2872, 728, 4741, 4911, 584, 879],
 [140, 4344, 4562, 1804],
 [497, 2788, 4737, 4193, 319, 144],
 [2517, 4911, 189, 3186, 2150, 1290, 4911, 1806, 2276, 3620],
 [1696,
  4813,
  4237,
  4124,
  2846,
  4277,
  1781,
  1990,
  4494,
  4159,
  2006,
  4598,
  2927,
  1517,
  879],
 [919, 3268, 4523, 1953, 2441, 1125, 16, 1481, 4424, 3383, 3211],
 [2490, 2091, 733, 1885, 4994, 2798, 4277, 2028, 4424, 3383, 3211],
 [1205, 3053, 1954, 4986, 3460, 1791, 28, 4002, 4277, 1569],
 [1131, 4413, 3110, 1070, 3395, 1844, 1600, 3609],
 [1614, 1121, 3119, 4196, 1803, 4650, 4546, 2999, 3510, 4028, 2783],
 [4193, 3521, 1072, 1791, 4277, 4994],
 [3550, 3566, 2581, 779, 3957, 4079, 179, 4398, 911],
 [1348, 2578, 4061, 780, 3875, 4614, 3761, 4424, 3383, 3211],
 [1133, 4549, 3542, 2941, 4601, 4424, 3383, 3211],
 [4645, 2533, 3257, 2037, 4252, 3871, 4032, 985, 861, 938],
 [4892, 3444, 2872],
 [2431, 3686, 4338, 2889, 4277, 3496, 3267, 879],


In [14]:
#Padding sequences
sent_length = 20
embedded_doc = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)

In [15]:
embedded_doc

array([[   0,    0,    0, ..., 1302, 1911, 1684],
       [   0,    0,    0, ..., 4911,  584,  879],
       [   0,    0,    0, ..., 4344, 4562, 1804],
       ...,
       [   0,    0,    0, ..., 4424, 3383, 3211],
       [   0,    0,    0, ..., 2859, 2044, 1703],
       [   0,    0,    0, ..., 3848, 2279,  447]], dtype=int32)

In [17]:
embedding_vectorfeatures = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vectorfeatures, input_length= sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [19]:
import numpy as np
X_final = np.array(embedded_doc)
y_final = np.array(y)

In [20]:
X_final.shape

(18285, 20)

In [21]:
y_final.shape

(18285,)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 42)

In [23]:
model.fit(X_train,y_train, validation_data = (X_test,y_test), epochs = 10, batch_size = 64)

Epoch 1/10
200/200 [==============================] - 14s 52ms/step - loss: 0.2910 - accuracy: 0.8632 - val_loss: 0.1994 - val_accuracy: 0.9171
Epoch 2/10
200/200 [==============================] - 10s 48ms/step - loss: 0.1355 - accuracy: 0.9487 - val_loss: 0.1965 - val_accuracy: 0.9152
Epoch 3/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0969 - accuracy: 0.9655 - val_loss: 0.2429 - val_accuracy: 0.9100
Epoch 4/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0694 - accuracy: 0.9778 - val_loss: 0.2826 - val_accuracy: 0.9050
Epoch 5/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0513 - accuracy: 0.9829 - val_loss: 0.2997 - val_accuracy: 0.9056
Epoch 6/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0332 - accuracy: 0.9895 - val_loss: 0.3468 - val_accuracy: 0.9112
Epoch 7/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0262 - accuracy: 0.9912 - val_loss: 0.3674 - val_accuracy:

In [24]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[2854,  253],
       [ 238, 2141]])

In [26]:
score = accuracy_score(y_test, y_pred)
score

0.9104994531534816